# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [76]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer





%matplotlib inline

In [33]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')

df = pd.read_sql_table('disaster_response', con=engine )
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [34]:
#we should remove the values that are not 0, and 1 in the 'related' column
df = df[df['related'] != 2]

In [36]:
Y['related'].value_counts()

1.0    19914
0.0     6101
Name: related, dtype: int64

In [31]:
Y.isnull().sum()

related                   138
request                   138
offer                     138
aid_related               138
medical_help              138
medical_products          138
search_and_rescue         138
security                  138
military                  138
child_alone               138
water                     138
food                      138
shelter                   138
clothing                  138
money                     138
missing_people            138
refugees                  138
death                     138
other_aid                 138
infrastructure_related    138
transport                 138
buildings                 138
electricity               138
tools                     138
hospitals                 138
shops                     138
aid_centers               138
other_infrastructure      138
weather_related           138
floods                    138
storm                     138
fire                      138
earthquake                138
cold      

Seems there are 138 NaN value in Y. I will use the mean strategy.

In [ ]:
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [97]:
def tokenize(text):
    """
    Tokenizes input text into individual words and lemmatizes each word.
    """
    tokens = word_tokenize(text.lower()) # Convert to lowercase and tokenize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if re.match(r'\w+', tok)] # Lemmatize each word and remove non-alphanumeric characters
    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [102]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, lowercase=True)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [100]:
# create a SimpleImputer object
imputer = SimpleImputer(strategy='mean')

# fit the imputer on X_train
imputer.fit(y_train)

# transform 
y_train = imputer.transform(y_train).round(0)
y_test = imputer.transform(y_test).round(0)



In [103]:
pipeline.fit(X_train, y_train)

/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9ab39473a0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                                       n_jobs=-1))])

In [104]:
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9ab39473a0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                         n_jobs=-1))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f9ab39473a0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                       n_jobs=-1),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocab

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [46]:
y_pred = pipeline.predict(X_test)

/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [73]:
def obtain_clf_report(y_pred, y_test):
    
    '''
    Function will produce the F1 score, precision, and recall for the model
    '''
    for col in range(y_test.shape[1]):
        print(f"Classification Report for {col}:")
        print(classification_report(y_pred[col], y_test[col],average='macro', zero_division='ignore'))

In [74]:
obtain_clf_report(y_pred, y_test)

Classification Report for 0:
              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97        35
         1.0       0.33      1.00      0.50         1

    accuracy                           0.94        36
   macro avg       0.67      0.97      0.74        36
weighted avg       0.98      0.94      0.96        36

Classification Report for 1:
              precision    recall  f1-score   support

         0.0       1.00      0.80      0.89        35
         1.0       0.12      1.00      0.22         1

    accuracy                           0.81        36
   macro avg       0.56      0.90      0.56        36
weighted avg       0.98      0.81      0.87        36

Classification Report for 2:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        35
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.

/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Moa

### 6. Improve your model
Use grid search to find better parameters. 

In [109]:
# update the pipeline to use the new classifier and preprocessor
# specify parameters for grid search 
parameters = {
     'clf__estimator__n_estimators': [20, 100],
     'clf__estimator__max_depth': [5, 20],
     'clf__estimator__min_samples_split': [2, 10]
}

# create grid search object 
cv = GridSearchCV(pipeline, param_grid=parameters)

cv






GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f9ab39473a0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__max_depth': [5, 20],
                         'clf__estimator__min_samples_split': [2, 10],
                         'clf__estimator__n_estimators': [20, 100]})

In [110]:
cv.fit(X_train, y_train)

/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' 

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f9ab39473a0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__max_depth': [5, 20],
                         'clf__estimator__min_samples_split': [2, 10],
                         'clf__estimator__n_estimators': [20, 100]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [113]:
cv_preds =cv.predict(X_test)

In [114]:
obtain_clf_report(cv_preds, y_test)

Classification Report for 0:
              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97        35
         1.0       0.33      1.00      0.50         1

    accuracy                           0.94        36
   macro avg       0.67      0.97      0.74        36
weighted avg       0.98      0.94      0.96        36

Classification Report for 1:
              precision    recall  f1-score   support

         0.0       1.00      0.80      0.89        35
         1.0       0.12      1.00      0.22         1

    accuracy                           0.81        36
   macro avg       0.56      0.90      0.56        36
weighted avg       0.98      0.81      0.87        36

Classification Report for 2:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        35
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.

/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Moaze002/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/Moa

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [115]:
import pickle
filename = 'model_RandomForrest.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.